In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS aviacao_gold;

CREATE OR REPLACE TABLE aviacao_gold.fato_bilhetes AS
WITH bilhetes_base AS (
    SELECT
        b.id                    AS bilhete_id,
        b.reserva_id            AS reserva_id,
        r.voo_id                AS voo_id,
        r.cliente_id            AS cliente_id,
        v.companhia_id          AS companhia_id,
        v.aeronave_id           AS aeronave_id,
        v.aeroporto_origem_id   AS aeroporto_origem_id,
        v.aeroporto_destino_id  AS aeroporto_destino_id,
        v.data_partida          AS data_partida_voo,
        v.status                AS status_voo,
        r.status                AS status_reserva,
        b.numero_bilhete        AS numero_bilhete,
        b.assento               AS assento,
        b.classe_cabine         AS classe_cabine,
        b.status_pagamento      AS status_pagamento,
        b.valor_tarifa          AS valor_tarifa,
        b.data_ref              AS data_ref_bilhete
    FROM aviacao_silver.dim_bilhetes b
    JOIN aviacao_silver.dim_reservas r
      ON b.reserva_id = r.id
     AND r.is_current = true
    JOIN aviacao_silver.dim_voos v
      ON r.voo_id = v.id
     AND v.is_current = true
    WHERE b.is_current = true
),
bagagens_agg AS (
    SELECT
        bag.bilhete_id,
        COUNT(*)             AS qtde_bagagens,
        SUM(bag.peso_kg)     AS peso_total_bagagens_kg
    FROM aviacao_silver.dim_bagagens bag
    WHERE bag.is_current = true
    GROUP BY bag.bilhete_id
)
SELECT
    b.*,
    COALESCE(bg.qtde_bagagens, 0)          AS qtde_bagagens,
    COALESCE(bg.peso_total_bagagens_kg, 0) AS peso_total_bagagens_kg
FROM bilhetes_base b
LEFT JOIN bagagens_agg bg
  ON b.bilhete_id = bg.bilhete_id;


CREATE OR REPLACE TABLE aviacao_gold.fato_voos_diario AS
SELECT
    f.voo_id,
    f.companhia_id,
    f.aeronave_id,
    f.aeroporto_origem_id,
    f.aeroporto_destino_id,
    CAST(f.data_partida_voo AS DATE)        AS data_voo,
    COUNT(DISTINCT f.bilhete_id)            AS qtde_bilhetes,
    SUM(f.valor_tarifa)                     AS receita_tarifas,
    SUM(f.qtde_bagagens)                    AS qtde_bagagens,
    SUM(f.peso_total_bagagens_kg)           AS peso_total_bagagens_kg,
    MAX(f.status_voo)                       AS status_voo
FROM aviacao_gold.fato_bilhetes f
GROUP BY
    f.voo_id,
    f.companhia_id,
    f.aeronave_id,
    f.aeroporto_origem_id,
    f.aeroporto_destino_id,
    CAST(f.data_partida_voo AS DATE);


CREATE OR REPLACE TABLE aviacao_gold.fato_manutencoes AS
SELECT
    m.id                          AS manutencao_id,
    m.aeronave_id                 AS aeronave_id,
    a.companhia_id                AS companhia_id,
    a.modelo_id                   AS modelo_id,
    m.tipo_manutencao             AS tipo_manutencao,
    m.data_inicio                 AS data_inicio,
    m.data_fim                    AS data_fim,
    DATEDIFF(DAY, m.data_inicio, m.data_fim) AS dias_manutencao,
    m.custo_total                 AS custo_total,
    m.data_ref                    AS data_ref_manutencao
FROM aviacao_silver.dim_manutencoes m
LEFT JOIN aviacao_silver.dim_aeronaves a
  ON m.aeronave_id = a.id
 AND a.is_current = true
WHERE m.is_current = true;


CREATE OR REPLACE VIEW aviacao_gold.vw_bilhetes_enriquecido AS
SELECT
    f.bilhete_id,
    f.reserva_id,
    f.voo_id,
    f.cliente_id,
    f.companhia_id,
    f.aeronave_id,
    f.aeroporto_origem_id,
    f.aeroporto_destino_id,
    f.data_partida_voo,
    f.status_voo,
    f.status_reserva,
    f.numero_bilhete,
    f.assento,
    f.classe_cabine,
    f.status_pagamento,
    f.valor_tarifa,
    f.qtde_bagagens,
    f.peso_total_bagagens_kg,

    cia.nome           AS companhia_nome,
    cia.codigo_iata    AS companhia_iata,
    cia.codigo_icao    AS companhia_icao,
    cia.pais           AS companhia_pais,
    cli.nome           AS cliente_nome,
    cli.pais           AS cliente_pais,
    ori.nome           AS aeroporto_origem_nome,
    ori.cidade         AS aeroporto_origem_cidade,
    ori.pais           AS aeroporto_origem_pais,
    dst.nome           AS aeroporto_destino_nome,
    dst.cidade         AS aeroporto_destino_cidade,
    dst.pais           AS aeroporto_destino_pais

FROM aviacao_gold.fato_bilhetes f
LEFT JOIN aviacao_silver.dim_companhias_aereas cia
  ON cia.id = f.companhia_id
 AND cia.is_current = true
LEFT JOIN aviacao_silver.dim_clientes cli
  ON cli.id = f.cliente_id
 AND cli.is_current = true
LEFT JOIN aviacao_silver.dim_aeroportos ori
  ON ori.id = f.aeroporto_origem_id
 AND ori.is_current = true
LEFT JOIN aviacao_silver.dim_aeroportos dst
  ON dst.id = f.aeroporto_destino_id
 AND dst.is_current = true;
